In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0,'/Users/physics/nanogw/Optimal-Statisitic-iPTA-Summer-2022')

In [2]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)
import numpy as np
import pickle
import json
import glob
import os
import math
import scipy


import matplotlib.pyplot as plt
%matplotlib inline


from enterprise.signals import signal_base
from enterprise.signals import gp_signals
from enterprise.pulsar import Pulsar
from enterprise.signals import parameter
from enterprise.signals import gp_priors
from enterprise.signals import utils

from enterprise_extensions import model_utils, blocks
from enterprise_extensions import model_orfs
from enterprise_extensions.frequentist import optimal_statistic as opt_stat
from enterprise.signals import selections
import enterprise_extensions.chromatic.chromatic



from enterprise_warp import enterprise_warp

import h5py
import la_forge.core as co
import ppta_dr2_models as dr2models

In [3]:
noisefile = '/Users/physics/nanogw/data/ppta_dr2_TN_noise.json'

with open(noisefile, 'r') as f:
    noisedict = json.load(f)

In [4]:
noisedict.keys()

dict_keys(['J0437-4715_0_magnexp__idx', 'J0437-4715_0_magnexp__log10_Amp', 'J0437-4715_0_magnexp__log10_tau', 'J0437-4715_0_magnexp__t0', 'J0437-4715_CASPSR_40CM_efac', 'J0437-4715_CASPSR_40CM_log10_tnequad', 'J0437-4715_CPSR2_10CM_efac', 'J0437-4715_CPSR2_10CM_log10_tnequad', 'J0437-4715_CPSR2_20CM_efac', 'J0437-4715_CPSR2_20CM_log10_tnequad', 'J0437-4715_CPSR2_50CM_efac', 'J0437-4715_CPSR2_50CM_log10_tnequad', 'J0437-4715_PDFB1_10CM_efac', 'J0437-4715_PDFB1_10CM_log10_tnequad', 'J0437-4715_PDFB1_1433_efac', 'J0437-4715_PDFB1_1433_log10_tnequad', 'J0437-4715_PDFB1_20CM_efac', 'J0437-4715_PDFB1_20CM_log10_tnequad', 'J0437-4715_PDFB1_early_10CM_efac', 'J0437-4715_PDFB1_early_10CM_log10_tnequad', 'J0437-4715_PDFB1_early_20CM_efac', 'J0437-4715_PDFB1_early_20CM_log10_tnequad', 'J0437-4715_PDFB_10CM_efac', 'J0437-4715_PDFB_10CM_log10_tnequad', 'J0437-4715_PDFB_20CM_efac', 'J0437-4715_PDFB_20CM_log10_tnequad', 'J0437-4715_PDFB_40CM_efac', 'J0437-4715_PDFB_40CM_log10_tnequad', 'J0437-4715_WB

In [5]:
noisedict
for ky,val in noisedict.items():
    if 'efac' in ky:
        if val > 2:
            print(ky,': ',val)

J0437-4715_CPSR2_20CM_efac :  2.3071643789004312
J0437-4715_PDFB1_1433_efac :  3.339282613815889
J0437-4715_PDFB1_20CM_efac :  2.640138126930894
J1824-2452A_WBCORR_10CM_efac :  2.4087329130418165
J1939+2134_CPSR2_50CM_efac :  2.9132216995503395


In [6]:
with open('/Users/physics/nanogw/data/PPTA_mock_pulsar_data.pkl','rb') as fin:
    psrs=pickle.load(fin)

In [7]:
for psr in psrs: 
    print(psr)

In [8]:
psrsname=[]
for p in psrs:
    psrsname.append(p.name)

print(psrsname)

['J0437-4715', 'J0613-0200', 'J0711-6830', 'J1017-7156', 'J1022+1001', 'J1024-0719', 'J1045-4509', 'J1125-6014', 'J1446-4701', 'J1545-4550', 'J1600-3053', 'J1603-7202', 'J1643-1224', 'J1713+0747', 'J1730-2304', 'J1732-5049', 'J1744-1134', 'J1824-2452A', 'J1832-0836', 'J1857+0943', 'J1909-3744', 'J1939+2134', 'J2124-3358', 'J2129-5721', 'J2145-0750', 'J2241-5236']


In [ ]:
"""
import os
 
# Function to rename multiple files
folder = "/Users/physics/Desktop/paramfilespta/"

p=0

for count, filename in enumerate(sorted(os.listdir(folder))):
    name=psrsname[count]
    add=name
    old=filename
    oldpath=folder+filename
    new=add+'_'+filename[5:] 
    newpath=folder+new
    print(newpath)
         
    # rename() function will
    # rename all the files
    os.rename(oldpath, newpath)
"""

In [ ]:
"""
keys = []
values = []
for k, v in noisedict.items():
    if 'J2241' in k: 
        keys.append(k)
        values.append(noisedict[k])
pulsar_noise=dict(zip(keys, values))
"""

In [ ]:
'''
with open("/Users/physics/Desktop/Noisepta/J2241noisefile.json", "w") as outfile:
    json.dump(pulsar_noise, outfile)
'''

In [9]:
class opts(object):pass
setattr(opts,'extra_model_terms','None')
setattr(opts,'job_config_xlsx',None)
setattr(opts,'num',0)
setattr(opts,'mpi_regime',None)
setattr(opts,'wipe_old_output',None)

In [10]:
len(psrs)

26

In [11]:
noisepath="/Users/physics/Desktop/Noisemodelpta/"
parampath="/Users/physics/Desktop/paramfilespta/"
full_model=[]
models=[]
point=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]

params = enterprise_warp.enterprise_warp.Params('/Users/physics/Desktop/ppta_gw/pta.txt', opts=opts)
pulsars = dr2models.PPTADR2Models(psrs,params)

for p in point:
    with open('/Users/physics/Desktop/ppta_gw/pta.json','r') as f:
        model_names = json.load(f)[psrs[p].name]
    model= pulsars.gwb()
    model += pulsars.efac()
    for ky in model_names.keys():
        print(ky)
        if ky != 'efac':
            if ky in ['system_noise','paired_ppta_band_noise','ppta_band_noise','fd_sys_g']:
                model +=getattr(pulsars,ky)(option=model_names[ky])
            else: 
                model += getattr(pulsars,ky)()
                
models.append(model)
full_model.append(model(psr))

------------------
Setting default parameters with file  /Users/physics/Desktop/ppta_gw/pta.txt
Setting default Solar System Ephemeris: DE436
Setting a default Enterprise clock convention (check the code)
Using all available pulsars from .par/.tim directory
Setting a default linear timing model
Including transient events to specific pulsar models
Setting reference radio frequency to 1400 MHz
------------------
Setting sampler kwargs from the parameter file:
Setting SCAMweight to 30
Setting AMweight to 15
Setting DEweight to 50
------------------
Current .par file:  J0437-4715.par
Current .tim file:  J0437-4715.tim
Number of Fourier frequencies for the GWB/CPL signal:  91
Adding HD ORF
efac
equad
ecorr
dm_noise


TypeError: no_selection() missing 1 required positional argument: 'toas'

In [ ]:
noisepath="/Users/physics/Desktop/Noisemodelpta/"
parampath="/Users/physics/Desktop/paramfilespta/"
point=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]
full_model=[]
models=[]

paramspta = enterprise_warp.enterprise_warp.Params('/Users/physics/Desktop/ppta_gw/pta.txt', opts=opts)
pulsars = dr2models.PPTADR2Models(psrs,paramspta)

for psr,param_file,noise_file,n in zip(psrs,sorted(os.listdir("/Users/physics/Desktop/paramfilespta/")),sorted(os.listdir("/Users/physics/Desktop/Noisemodelpta")),point): 
    model= pulsars.gwb()
    pnm = psr.name
    print(noisepath+noise_file)
    with open(noisepath+noise_file,'r') as f:
        model_names = json.load(f)[pnm]
    opts.num=n
    print(parampath+param_file)
    params = enterprise_warp.enterprise_warp.Params(parampath+param_file, opts=opts)
    pulsar = dr2models.PPTADR2Models(psr,params)
    model += pulsar.efac()
    for ky in model_names.keys():
        print(ky)
        if ky != 'efac':
            if ky in ['system_noise','paired_ppta_band_noise','ppta_band_noise','fd_sys_g']:
                model +=getattr(pulsar,ky)(option=model_names[ky])
            else: 
                model += getattr(pulsar,ky)()
                
    models.append(model)
        
    #with open(f'path/to/pickled/models/{pnm}_model_psr.pkl','wb') as fout:
     #   pickle.dump(full_model,fout)

In [11]:
##Working Model
noisepath="/Users/physics/nanogw/Optimal-Statisitic-iPTA-Summer-2022/Noisemodelpta/"
parampath="/Users/physics/nanogw/Optimal-Statisitic-iPTA-Summer-2022/paramfilespta/"
point=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]
full_model=[]
models=[]

Tspan = model_utils.get_tspan(psrs)

gw = blocks.common_red_noise_block(psd='powerlaw', prior='log-uniform', Tspan=Tspan, components=30, gamma_val=4.33, name='gw')
for psr,param_file,noise_file,n in zip(psrs,sorted(os.listdir("/Users/physics/nanogw/Optimal-Statisitic-iPTA-Summer-2022/paramfilespta/")),sorted(os.listdir("/Users/physics/nanogw/Optimal-Statisitic-iPTA-Summer-2022/Noisemodelpta/")),point): 
    pnm = psr.name
    print(noisepath+noise_file)
    with open(noisepath+noise_file,'r') as f:
        model_names = json.load(f)[pnm]
    opts.num=n
    print(parampath+param_file)
    params = enterprise_warp.enterprise_warp.Params(parampath+param_file, opts=opts)
    pulsar = dr2models.PPTADR2Models(psr,params)
    model = gw
    model += pulsar.efac()
    for ky in model_names.keys():
        print(ky)
        if ky != 'efac':
            if ky in ['system_noise','paired_ppta_band_noise','ppta_band_noise','fd_sys_g']:
                model +=getattr(pulsar,ky)(option=model_names[ky])
            else: 
                model += getattr(pulsar,ky)()
                
    models.append(model)
        
    #with open(f'path/to/pickled/models/{pnm}_model_psr.pkl','wb') as fout:
     #   pickle.dump(full_model,fout)

/Users/physics/nanogw/Optimal-Statisitic-iPTA-Summer-2022/Noisemodelpta/J0437-4715_noisemodel.json
/Users/physics/nanogw/Optimal-Statisitic-iPTA-Summer-2022/paramfilespta/J0437-4715_param_fixed_whitenoise.txt
------------------
Setting default parameters with file  /Users/physics/nanogw/Optimal-Statisitic-iPTA-Summer-2022/paramfilespta/J0437-4715_param_fixed_whitenoise.txt
Setting default Solar System Ephemeris: DE436
Setting a default Enterprise clock convention (check the code)
Using all available pulsars from .par/.tim directory
Setting a default linear timing model
Including transient events to specific pulsar models
Setting reference radio frequency to 1400 MHz
------------------
Setting sampler kwargs from the parameter file:
Setting SCAMweight to 30
Setting AMweight to 15
Setting DEweight to 50
------------------
Current .par file:  J0437-4715.par
Current .tim file:  J0437-4715.tim
efac
equad
spin_noise
dm_noise
j0437_event
fd_sys_g
ppta_band_noise
paired_ppta_band_noise
system_

Current .par file:  J1446-4701.par
Current .tim file:  J1446-4701.tim
efac
equad
spin_noise
dm_noise
/Users/physics/nanogw/Optimal-Statisitic-iPTA-Summer-2022/Noisemodelpta/J1545-4550_noisemodel.json
/Users/physics/nanogw/Optimal-Statisitic-iPTA-Summer-2022/paramfilespta/J1545-4550_param_fixed_whitenoise.txt
------------------
Setting default parameters with file  /Users/physics/nanogw/Optimal-Statisitic-iPTA-Summer-2022/paramfilespta/J1545-4550_param_fixed_whitenoise.txt
Setting default Solar System Ephemeris: DE436
Setting a default Enterprise clock convention (check the code)
Using all available pulsars from .par/.tim directory
Setting a default linear timing model
Including transient events to specific pulsar models
Setting reference radio frequency to 1400 MHz
------------------
Setting sampler kwargs from the parameter file:
Setting SCAMweight to 30
Setting AMweight to 15
Setting DEweight to 50
------------------
Current .par file:  J1545-4550.par
Current .tim file:  J1545-4550.t

Current .par file:  J1824-2452A.par
Current .tim file:  J1824-2452A.tim
efac
equad
spin_noise
dm_noise
paired_ppta_band_noise
/Users/physics/nanogw/Optimal-Statisitic-iPTA-Summer-2022/Noisemodelpta/J1832-0836_noisemodel.json
/Users/physics/nanogw/Optimal-Statisitic-iPTA-Summer-2022/paramfilespta/J1832-0836_param_fixed_whitenoise.txt
------------------
Setting default parameters with file  /Users/physics/nanogw/Optimal-Statisitic-iPTA-Summer-2022/paramfilespta/J1832-0836_param_fixed_whitenoise.txt
Setting default Solar System Ephemeris: DE436
Setting a default Enterprise clock convention (check the code)
Using all available pulsars from .par/.tim directory
Setting a default linear timing model
Including transient events to specific pulsar models
Setting reference radio frequency to 1400 MHz
------------------
Setting sampler kwargs from the parameter file:
Setting SCAMweight to 30
Setting AMweight to 15
Setting DEweight to 50
------------------
Current .par file:  J1832-0836.par
Current

In [12]:
#import cloudpickle
#with open('/Users/physics/nanogw/data/ppta_models.pkl','wb') as fout:
 #   cloudpickle.dump(models,fout)

In [13]:
for p in psrs:
    p.name=p.name+'_ppta'
for p in psrs:
    print(p.name)

J0437-4715_ppta
J0613-0200_ppta
J0711-6830_ppta
J1017-7156_ppta
J1022+1001_ppta
J1024-0719_ppta
J1045-4509_ppta
J1125-6014_ppta
J1446-4701_ppta
J1545-4550_ppta
J1600-3053_ppta
J1603-7202_ppta
J1643-1224_ppta
J1713+0747_ppta
J1730-2304_ppta
J1732-5049_ppta
J1744-1134_ppta
J1824-2452A_ppta
J1832-0836_ppta
J1857+0943_ppta
J1909-3744_ppta
J1939+2134_ppta
J2124-3358_ppta
J2129-5721_ppta
J2145-0750_ppta
J2241-5236_ppta


In [14]:
len(models)

26

In [15]:
len(psrs)

26

In [16]:
full_model2=[]
for m,psr in zip(models,psrs):
    full_model2.append(m(psr))

In [17]:
noisefile = '/Users/physics/nanogw/data/ppta_dr2_TN_noise.json'

with open(noisefile, 'r') as f:
    noisedict = json.load(f)

In [18]:
noisedictcp=noisedict.copy()
#for ky,val in noisedict.items():
#    if 'equad'in ky and not 'tnequad' in ky:
#        kynew=ky.replace('equad','tnequad')
#        noisedictcp[kynew] = noisedict[ky]

noisedictcp2=noisedictcp.copy()
for ky,val in noisedictcp.items():
    split=ky.split('_')
    split.insert(1,'ppta')
    kynew='_'.join(split)
    noisedictcp2[kynew] = noisedictcp[ky]
for ky,val in noisedictcp.items():
    if ky in noisedictcp:
        noisedictcp2.pop(ky)

In [19]:
pta = signal_base.PTA(full_model2)
# We need to set the white noise parameters to the values in the noise dictionary
pta.set_default_params(noisedictcp2)

In [20]:
# Load up the maximum-likelihood values for the pulsars' red noise parameters and the common red process
# These values come from the results of a Bayesian search

with open("/Users/physics/nanogw/data/ppta_ml_params.pkl",'rb') as fin:
    ml_params=pickle.load(fin)

ml_paramscp=ml_params.copy()
for ky,val in ml_params.items():
    split=ky.split('_')
    split.insert(1,'ppta')
    kynew='_'.join(split)
    ml_paramscp[kynew] = ml_params[ky]
for ky,val in ml_params.items():
    if ky in ml_params:
        ml_paramscp.pop(ky)
ml_paramscp['gw_log10_A']=ml_paramscp['gw_ppta_log10_A']

In [21]:
ml_paramscp['J0437-4715_ppta_chromatic_gp_idx']=4
ml_paramscp['J0613-0200_ppta_chromatic_gp_idx']=4
ml_paramscp['J1017-7156_ppta_chromatic_gp_idx']=2.39
ml_paramscp['J1045-4509_ppta_chromatic_gp_idx']=1.82
ml_paramscp['J1939+2134_ppta_chromatic_gp_idx']=4

In [22]:
for ky in ml_paramscp.keys():
    if ky not in pta.param_names:
        print(ky)

gw_ppta_log10_A
nmodel_ppta
lnpost_ppta
lnlike_ppta
chain_ppta_accept
pt_ppta_chain_accept


In [23]:
class OS:
    def __init__(self, psrs, pta, params):
        self.psrs, self.params = psrs, params
        self.pairs  = [(i,j) for i in range(len(pta)) for j in range(i+1, len(pta))]
        self.angles = np.array([np.arccos(np.dot(self.psrs[i].pos, self.psrs[j].pos)) for (i,j) in self.pairs])
        
        ys = [sm._residuals for sm in pta]
        Cs = [Cmat(sm, params) for sm in pta]

        Fgws  = [sm['gw'].get_basis(params) for sm in pta]
        self.Gamma = np.array(pta[0]['gw'].get_phi({**params, 'gw_log10_A': 0})).copy()
        assert self.Gamma.ndim == 1

        FCys = [C.solve(y, Fgw) for C, y, Fgw in zip(Cs, ys, Fgws)]
        self.FCFs = [C.solve(Fgw, Fgw) for C, Fgw in zip(Cs, Fgws)]
        
        # a . np.diag(g) . b = a . (g * b) = (a * g) . b
        ts = [np.dot(FCys[i], self.Gamma * FCys[j]) for (i,j) in self.pairs]
        # A . np.diag(g) . B = (A * g) . B
        bs = [np.trace(np.dot(self.FCFs[i] * self.Gamma, self.FCFs[j] * self.Gamma)) for (i,j) in self.pairs]
        
        self.rhos   = np.array(ts) / np.array(bs)
        self.sigmas = 1.0 / np.sqrt(bs)
    
    def _set_orf(self, orf):
        if not hasattr(self, '_orf') or self._orf is not orf:
            self.orfs = np.array([orf(self.psrs[i].pos, self.psrs[j].pos) for (i,j) in self.pairs])        
            self._orf = orf
    
    def os(self, orf=utils.hd_orf, sel=(lambda p: slice(None))):
        self._set_orf(orf)
        
        mask = sel(self.pairs)
        return (np.sum(self.rhos[mask] * self.orfs[mask] / self.sigmas[mask]**2) /
                np.sum(self.orfs[mask]**2 / self.sigmas[mask]**2))
    
    def os_sigma(self, orf=utils.hd_orf, sel=(lambda p: slice(None))):
        self._set_orf(orf)

        mask = sel(self.pairs)
        return 1.0 / np.sqrt(np.sum(self.orfs[mask]**2 / self.sigmas[mask]**2))
    
    def gw_mean(self):
        return np.array([10**(2.0 * self.params['gw_log10_A'])] * len(self.pairs))
    
    def _tracedot(self, orf, *args):        
        ret = np.identity(len(self.Gamma))
    
        for i, (j,k) in zip(args[::2], args[1::2]):
            ret = np.dot(ret, self.FCFs[i] * self.Gamma * orf(self.psrs[j].pos, self.psrs[k].pos))
        
        return np.trace(ret)
    
    def gw_corr(self, orf=utils.hd_orf):
        Agw = 10**self.params['gw_log10_A']
        self.orfs = np.array([orf(self.psrs[i].pos, self.psrs[j].pos) for (i,j) in self.pairs])
        
        sigma = np.zeros((len(self.pairs), len(self.pairs)), 'd')
        
        for ij in range(len(self.pairs)):
            i, j = self.pairs[ij]
            
            for kl in range(ij, len(self.pairs)):
                k, l = self.pairs[kl]
                
                if ij == kl:
                    sigma[ij, kl] = (self._tracedot(orf, i, (i,j), j, (j,i)) + 
                                     Agw**4 * self._tracedot(orf, i, (i,j), j, (j,i), i, (i,j), j, (j,i)))
                
                    # ijij -> ij ij + ii jj + ij ji
                    # iCGCj iCGCj -> A^4 (CGCG)^2 + iCGCj jCGCi + A^4 iCGCj iCGCj
                
                elif i == k and j != l:
                    sigma[ij, kl] = (Agw**2 * self._tracedot(orf, i, (i,j), j, (j,l), l, (l,i)) +
                                     Agw**4 * self._tracedot(orf, i, (i,j), j, (j,i), i, (i,l), l, (l,i)))

                    # ijil -> ij il + ii jl + il ji
                    # iCGCj iCGCl -> A^4 (CGCG)^2 + A^2 iCGCj lCGCi + A^4 iCGCj iCGCl
    
                elif i != k and j == l:
                    sigma[ij, kl] = (Agw**2 * self._tracedot(orf, j, (j,k), k, (k,i), i, (i,j)) +
                                     Agw**4 * self._tracedot(orf, i, (i,j), j, (j,k), k, (k,j), j, (j,i)))
                    
                    # ijkj -> ij kj + ik jj + ij jk
                    # iCGCj kCGCj -> A^4 (CGCG)^2 + A^2 jCGCk iCGCj + A^4 iCGCj kCGCj
                    
                elif i != k and j != l and i != l and j != k:
                    sigma[ij, kl] = Agw**4 * (self._tracedot(orf, i, (i,j), j, (j,k), k, (k,l), l, (l,i)) + 
                                              self._tracedot(orf, i, (i,j), j, (j,l), l, (l,k), k, (k,i)))                    
                    
                    # ijkl -> ij kl + ik jl + il jk
                    # iCGCj kCGCl -> A^4 (CGCG)^2 + A^4 iCGCj lCGCk + A^4 iCGCj kCGCl

                sigma[ij, kl] = sigma[ij, kl] * self.sigmas[ij]**2 * self.sigmas[kl]**2
                sigma[kl, ij] = sigma[ij, kl]
                
        return sigma
                    
    def snr(self, orf=utils.hd_orf, sel=(lambda p: slice(None))):
        return self.os(orf, sel) / self.os_sigma(orf, sel)


class Cmat:
    def __init__(self, sm, params):
        self.Nmat = sm.get_ndiag(params)
        # try:
        #         cf = sl.cho_factor(Sigma)
        #         SigmaTNr = sl.cho_solve(cf, TNr)
        #         SigmaTNF = sl.cho_solve(cf, FNT.T)
        #     except np.linalg.LinAlgError:
        #         SigmaTNr = np.linalg.solve(Sigma, TNr)
        #         SigmaTNF = np.linalg.solve(Sigma, FNT.T)
        self.Sigma = scipy.linalg.cho_factor(sm.get_TNT(params) + np.diag(sm.get_phiinv(params)),check_finite=False)
        self.Fmat = sm.get_basis(params)                       
    
    def solve(self, r, l):
        FNr = self.Nmat.solve(r, self.Fmat)
        FNl = FNr if (l is r) else self.Nmat.solve(l, self.Fmat)
        
        return self.Nmat.solve(l, r) - np.dot(FNl.T, scipy.linalg.cho_solve(self.Sigma, FNr))

In [24]:
optimal_stat=OS(psrs,pta,ml_paramscp)

In [25]:
OS.os(optimal_stat)

1.273550140751694e-27

In [26]:
OS.snr(optimal_stat)

1470.5067713071396

In [ ]:
ostat = opt_stat.OptimalStatistic(psrs, pta=pta, orf='hd')
ostat_dip = opt_stat.OptimalStatistic(psrs, pta=pta, orf='dipole')
ostat_mono = opt_stat.OptimalStatistic(psrs, pta=pta, orf='monopole')

In [ ]:
# Compute the optimal statistic
# The optimal statistic returns five quantities:
#  - xi: an array of the angular separations between the pulsar pairs (in radians)
#  - rho: an array of the cross-correlations between the pulsar pairs
#  - sig: an array of the uncertainty in the cross-correlations
#  - OS: the value of the optimal statistic
#  - OS_sig: the uncertainty in the optimal statistic

xi, rho, sig, OS, OS_sig = ostat.compute_os(params=ml_paramscp) #HD
print(OS, OS_sig, OS/OS_sig)

_, _, _, OS_dip, OS_sig_dip = ostat_dip.compute_os(params=ml_paramscp) #Dipole
print(OS_dip, OS_sig_dip, OS_dip/OS_sig_dip)

_, _, _, OS_mono, OS_sig_mono = ostat_mono.compute_os(params=ml_paramscp) #Monopole 
print(OS_mono, OS_sig_mono, OS_mono/OS_sig_mono)

In [ ]:
ml_paramscp

In [ ]:
ml_params
for ky,val in ml_params.items():
    if 'red_noise_log10_A' in ky:
        if val > (10**-6):
            print(ky,': ',val)

In [ ]:
ml_params
for ky,val in ml_params.items():
    if 'red_noise_log10_A' in ky:
        print(ky,': ',val)

In [ ]:
for nm in pta.param_names:
    if nm not in ml_paramscp.keys():
        print(nm)

In [ ]:
#ppta_param_names=[]
#for name in ml_paramscp.keys(): 
 #   ppta_param_names.append(name)

In [ ]:
#len(ppta_param_names)

In [ ]:
ppta_param_names=np.loadtxt('/Users/physics/nanogw/data/chains_ppta_dr2/pars.txt',dtype=str)

In [ ]:
ppta_param_namescp=[]
for ky in ppta_param_names:
    split=ky.split('_')
    split.insert(1,'ppta')
    kynew='_'.join(split)
    ppta_param_namescp.append(kynew)
idx=ppta_param_namescp.index('gw_ppta_log10_A')
ppta_param_namescp[idx]='gw_log10_A'

In [ ]:
ppta_param_namescp

In [ ]:
len(pta.param_names)

In [ ]:
for nm in pta.param_names:
    if nm not in ppta_param_namescp :
        print(nm)

In [ ]:
len(ppta_param_namescp)

In [ ]:
type(ppta_param_names)

In [ ]:
ppta_array=np.load("/Users/physics/nanogw/data/combined_ppta_chain.npy") 

In [ ]:
ppta_array.shape

In [ ]:
ppta_array[:,-5]

In [ ]:
plt.hist(ppta_array[:,-5],bins=[-.5,.5,1.5])
plt.show()

In [ ]:
if (np.isnan(ppta_array).any()): 
    print("NAN")

In [ ]:
if (np.isinf(ppta_array).any()):
    print("inf")

In [ ]:
ahat= []
snr = []
for ii in range(10):
    param =(dict(zip(ppta_param_namescp,ppta_array[ii,:-4])))
    param['J0437-4715_ppta_chromatic_gp_idx']=4
    param['J0613-0200_ppta_chromatic_gp_idx']=4
    param['J1017-7156_ppta_chromatic_gp_idx']=2.39
    param['J1045-4509_ppta_chromatic_gp_idx']=1.82
    param['J1939+2134_ppta_chromatic_gp_idx']=4
    xi, rho, sig, Ahat, Ahat_err = ostat.compute_os(params=param) #HD
    ahat.append(Ahat)
    snr.append(Ahat/Ahat_err)
    print(Ahat/Ahat_err)

In [ ]:
pta.get_lnlikelihood(ml_paramscp)

In [ ]:
ml_params['lnlike']

In [ ]:
pta.get_lnprior(ml_paramscp)

In [ ]:
Tspan